In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!cp -r /kaggle/input/qwen3-4b-ft-eval-overture/qwen3-4b-qlora-overture/qwen3-4b-qlora-overture /kaggle/working/

In [4]:
!cp /kaggle/input/qwen3-4b-ft-eval-overture/overture_cleaned_places.csv /kaggle/working/
!cp /kaggle/input/qwen3-4b-ft-eval-overture/make_sft_jsonl.py /kaggle/working/

In [5]:
!python make_sft_jsonl.py

✅ Done. Samples: 6000  → train/val/test = 4200/600/1200
Example:
{
  "instruction": "Decide whether the two place records refer to the same real-world place. Answer ONLY 'YES' or 'NO'.",
  "input": "Record A: Condomínio Edifício Gramado | landmark_and_historical_building | Rua Antônio de Barros, 2526 | 551129419896.0\nRecord B: Condomínio Edifício Antúrio | Community and Government > Residential Building > Apartment or Condo | Rua Reboujo, 250 | (11) 2091-3330",
  "output": "NO"
}
{
  "instruction": "Decide whether the two place records refer to the same real-world place. Answer ONLY 'YES' or 'NO'.",
  "input": "Record A: Havanna Inh. Hüseyen Özcelik Cocktailbar | Dining and Drinking > Bar | Holwedestr. 1 | 04953 18868947 | https://www.havannarestaurant.de\nRecord B: Havanna | restaurant | Holwedestraße 1 | 495318868947.0 | http://www.havanna-restaurant.de",
  "output": "YES"
}


In [6]:
pip install trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 11.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 95.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 38.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 80.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 29.3 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import json, re, time
from collections import Counter

from datasets import load_dataset
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    classification_report,
    confusion_matrix,
    roc_auc_score,
    average_precision_score,
    roc_curve,
    precision_recall_curve,
)

import matplotlib.pyplot as plt

# ---------------- 基本配置 ----------------
BASE = "Qwen/Qwen3-4B-Instruct-2507"
ADAPTER = "qwen3-4b-qlora-overture"

bnb = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

tok = AutoTokenizer.from_pretrained(BASE, use_fast=True, trust_remote_code=True)
tok.pad_token = tok.eos_token
tok.padding_side = "left"

base_model = AutoModelForCausalLM.from_pretrained(
    BASE,
    quantization_config=bnb,
    device_map="auto",
    trust_remote_code=True
)
model = PeftModel.from_pretrained(base_model, ADAPTER)

print("Active adapter:", getattr(model, "active_adapter", None))

model.eval()
torch.set_grad_enabled(False)

# 预先拿到 YES / NO 对应的 token id（用在概率计算里）
YES_TOK_ID = tok(" YES", add_special_tokens=False)["input_ids"][-1]
NO_TOK_ID  = tok(" NO",  add_special_tokens=False)["input_ids"][-1]


# ---------------- 文本预测（保持你原来的逻辑，方便打印样本） ----------------
def predict_yesno_text(prompt, max_new_tokens=2):
    """生成完整文本，并从中解析 YES/NO，返回 (label_str, full_text)."""
    inputs = tok(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            temperature=0.0,
            top_p=1.0
        )
    text = tok.decode(out[0], skip_special_tokens=True)

    post = text.split("Answer:")[-1].strip()
    first = post.split()[0].strip(",.?!:;").upper() if post else ""

    mapping_yes = {"YES", "1", "TRUE"}
    mapping_no  = {"NO", "0", "FALSE"}

    if first in mapping_yes:
        return "YES", text
    if first in mapping_no:
        return "NO", text

    # 兜底：全句再搜一遍
    m = re.search(r"\b(YES|NO|1|0|TRUE|FALSE)\b", text.upper())
    if m:
        tok_ = m.group(1)
        return ("YES" if tok_ in mapping_yes else "NO"), text

    return "NO", text


# ---------------- 概率打分（用来算 AUC / PR-AUC） ----------------
def score_yes_probability(prompt):
    """
    不生成整段文本，只做一次前向，取最后一个位置对 "YES"/"NO" 的概率，
    返回: (pred_label_int, yes_score_float)
    """
    # 这里直接在 prompt 后加一个空格，模型会在最后一个 token 位置输出分布
    enc = tok(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model(**enc)
        logits = out.logits[:, -1, :]  # [batch=1, vocab]
        probs = torch.softmax(logits, dim=-1)[0]

    p_yes = probs[YES_TOK_ID].item()
    p_no  = probs[NO_TOK_ID].item()

    # 归一化一下，只在 YES/NO 两个选项里归一
    denom = p_yes + p_no + 1e-9
    p_yes_norm = p_yes / denom

    pred_label = 1 if p_yes_norm >= 0.5 else 0
    return pred_label, p_yes_norm


# ---------------- 读取测试集 ----------------
ds = load_dataset("json", data_files="sft_data/test.jsonl")["train"]

y_true = []
y_pred = []        # 最终离散预测 (0/1)
y_scores = []      # YES 的概率，用于 AUC / PR-AUC

PRINT_N = 10
printed = 0

start_time = time.perf_counter()

for idx, ex in enumerate(ds):
    prompt = f"{ex['instruction']}\n{ex['input']}\nAnswer:"

    # 1）先用概率打分，后面指标都用这两个东西：
    #   - pred_label: 0/1
    #   - yes_score: float
    pred_label, yes_score = score_yes_probability(prompt)
    y_scores.append(yes_score)
    y_pred.append(pred_label)

    gold_label = 1 if ex["output"].strip().upper() == "YES" else 0
    y_true.append(gold_label)

    # 2）前几条样本仍然打印完整生成，方便你人工检查
    if printed < PRINT_N:
        label_str, raw = predict_yesno_text(prompt)
        print(f"\n===== SAMPLE #{idx} =====")
        print("PROMPT:\n", prompt)
        print("RAW GENERATION:\n", raw)
        print("PARSED (from text):", label_str,
              "   PROB(YES) from score_yes_probability():", yes_score,
              "   GOLD:", "YES" if gold_label == 1 else "NO")
        printed += 1

end_time = time.perf_counter()
total_time = end_time - start_time
num_samples = len(y_true)
avg_latency = total_time / num_samples
time_per_1000 = avg_latency * 1000.0

# ---------------- 经典分类指标 ----------------
acc = accuracy_score(y_true, y_pred)
p, r, f1, _ = precision_recall_fscore_support(
    y_true, y_pred, average="binary", zero_division=0
)

print("\n========== BASIC METRICS ==========")
print("Label balance (gold):", Counter(y_true))
print("Pred distribution:", Counter(y_pred))
print("Confusion matrix:\n", confusion_matrix(y_true, y_pred))
print("\nClassification report:")
print(classification_report(y_true, y_pred, digits=4))
print(f"Acc={acc:.4f}  P={p:.4f}  R={r:.4f}  F1={f1:.4f}")

# ---------------- AUC / PR-AUC + 曲线 ----------------
print("\n========== AUC / PR-AUC ==========")

try:
    roc_auc = roc_auc_score(y_true, y_scores)
    pr_auc = average_precision_score(y_true, y_scores)
    print(f"ROC-AUC = {roc_auc:.4f}")
    print(f"PR-AUC  = {pr_auc:.4f}")
except ValueError as e:
    print("Cannot compute AUC/PR-AUC (maybe only one class present):", e)
    roc_auc = None
    pr_auc = None

# ROC 曲线
try:
    fpr, tpr, _ = roc_curve(y_true, y_scores)
    plt.figure()
    plt.plot(fpr, tpr, label=f"ROC (AUC = {roc_auc:.4f})")
    plt.plot([0, 1], [0, 1], linestyle="--")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("Qwen3-4B-Instruct (FT) – ROC Curve")
    plt.legend(loc="lower right")
    plt.tight_layout()
    plt.savefig("qwen3_4b_ft_roc_curve.png")
    plt.close()
    print("Saved ROC curve to qwen3_4b_ft_roc_curve.png")
except Exception as e:
    print("Error plotting ROC curve:", e)

# PR 曲线
try:
    precision, recall, _ = precision_recall_curve(y_true, y_scores)
    plt.figure()
    plt.plot(recall, precision, label=f"PR (AP = {pr_auc:.4f})")
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.title("Qwen3-4B-Instruct (FT) – Precision-Recall Curve")
    plt.legend(loc="lower left")
    plt.tight_layout()
    plt.savefig("qwen3_4b_ft_pr_curve.png")
    plt.close()
    print("Saved PR curve to qwen3_4b_ft_pr_curve.png")
except Exception as e:
    print("Error plotting PR curve:", e)

# ---------------- Latency ----------------
print("\n========== LATENCY ==========")
print(f"Total samples   : {num_samples}")
print(f"Total time (s)  : {total_time:.4f}")
print(f"Avg latency (s/sample): {avg_latency:.6f}")
print(f"Time per 1000 samples (s): {time_per_1000:.4f}")

2025-11-24 08:45:57.183229: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763973957.397220     128 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763973957.457788     128 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

Active adapter: default


Generating train split: 0 examples [00:00, ? examples/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



===== SAMPLE #0 =====
PROMPT:
 Decide whether the two place records refer to the same real-world place. Answer ONLY 'YES' or 'NO'.
Record A: The Bristol Hotel | resort | 115 Country Music Way | 2766963535.0 | https://www.bristolhotelva.com/?utm_medium=organic&utm_source=yext&utm_campaign=website
Record B: The Bristol Hotel | resort | 510 Birthplace Of Country Music Way | 2766963535 | https://www.bristolhotelva.com/
Answer:
RAW GENERATION:
 Decide whether the two place records refer to the same real-world place. Answer ONLY 'YES' or 'NO'.
Record A: The Bristol Hotel | resort | 115 Country Music Way | 2766963535.0 | https://www.bristolhotelva.com/?utm_medium=organic&utm_source=yext&utm_campaign=website
Record B: The Bristol Hotel | resort | 510 Birthplace Of Country Music Way | 2766963535 | https://www.bristolhotelva.com/
Answer: YES
PARSED (from text): YES    PROB(YES) from score_yes_probability(): 0.9955311303414716    GOLD: YES

===== SAMPLE #1 =====
PROMPT:
 Decide whether the two p